In [94]:
%load_ext autoreload
%autoreload 2

In [111]:
# import needed external libraries
import os
import sys
import pandas as pd
import string, re
from datetime import datetime
from datetime import timedelta

import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
import sklearn

# Include the `src` module in the module path for imports
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.functions import prep

### Import Data

In [120]:
df_comments = pd.read_csv('../../src/data/csv_exports/comments.csv')

df_trivia = pd.read_csv('../../src/data/csv_exports/trivia.csv')

df_metadata = pd.read_csv('../../src/data/csv_exports/metadata.csv')

df_brands = pd.read_csv('../../src/data/csv_exports/brands.csv')

### Filter to list of actual answerable questions

In [123]:
# remove comments that don't have '?'
df_comments = df_comments[df_comments['comment_text'].str.contains('\?')]

# remove questions from staff members
q_c = df_comments[df_comments['comment_flag'] == 0]

# remove questions in a reply comment and reset indexes
q_c = q_c[q_c['parent_id'] == 0]
q_c = q_c.reset_index(drop=True)

# remove questions with more than one '?' and reset indexes
for i, q in enumerate(q_c['comment_text']):
    if q.count('?') > 1:
        q_c = q_c.drop(i)
q_c = q_c.reset_index(drop=True)

# remove questions with '?!' or '!?' and reset indexes
for i, q in enumerate(q_c['comment_text']):
    if q.count('!?') > 0:
        q_c = q_c.drop(i)
q_c = q_c.reset_index(drop=True)

for i, q in enumerate(q_c['comment_text']):
    if q.count('?!') > 0:
        q_c = q_c.drop(i)
q_c = q_c.reset_index(drop=True)

# remove questions asked on Friday Live category articles


(25640, 8)
(25435, 8)
(16538, 8)
(12248, 8)
(11855, 8)


In [122]:
q_c

,article_num,comment_id,commenter_name,comment_published,comment_text,comment_likes,comment_flag,parent_id
0,3605,24,Ninja2789,2017-01-11T16:55:57.000Z,FINALLY! comments on the app. I love you guy...,7,0,0
1,3605,26,Snake.Plissken,2017-01-11T16:58:41.000Z,Is it just me or has the update seriously affe...,1,0,0
2,3605,41,R.Francis,2017-01-11T17:22:05.000Z,Is it only possible to view and comment on art...,1,0,0
3,3605,49,bartc,2017-01-11T17:41:34.000Z,I'm not clear. Can this be only accessed on iO...,1,0,0
4,3605,78,Edson,2017-01-11T18:42:25.000Z,Great feat. Is there a way to link our newly c...,1,0,0
...,...,...,...,...,...,...,...,...
11850,6525,146399,GTVeloce,2020-04-07T10:03:01.000Z,Unless it's the actor's own personal watch it'...,0,0,0
11851,6527,146634,Stickmansa,2020-04-07T21:50:09.000Z,How long to people think the Alu Bezel will last?,0,0,0
11852,6429,146672,socrates35,2020-04-07T23:00:32.000Z,"Fascinating watch, with a gasp-inducing price ...",0,0,0
11853,6520,146732,mrfreaz,2020-04-08T03:54:36.000Z,Nice Movado Homage. But aren't homages suppose...,0,0,0


In [124]:
q_c['article_num'].value_counts()

4202    179
4251    140
4214     44
4306     37
5414     36
       ... 
5577      1
4957      1
5553      1
5441      1
4094      1
Name: article_num, Length: 2441, dtype: int64

In [129]:
df_metadata[df_metadata['article_id'] == 4306]['article_cat']

4306    Friday Live
Name: article_cat, dtype: object

In [113]:
(datetime.strptime(q_c['comment_published'][0][:-5], '%Y-%m-%dT%H:%M:%S') - datetime.strptime(df_metadata['article_published'][q_c['article_num'][0]][:-5], '%Y-%m-%d %H:%M:%S-')) > timedelta(20515)

False

In [114]:
type(datetime.strptime(q_c['comment_published'][0][:-5], '%Y-%m-%dT%H:%M:%S') - datetime.strptime(df_metadata['article_published'][q_c['article_num'][0]][:-5], '%Y-%m-%d %H:%M:%S-'))

datetime.timedelta

In [119]:
df_metadata

,article_id,article_cat,article_title,article_author,article_published,article_modified,comm_count,word_count
0,0,Not Specified,Introducing Three Limited Edition H. Mosers wi...,HODINKEE,2013-12-09 10:35:43-05:00,2017-02-20 21:06:01-05:00,-42,246
1,1,Not Specified,"Come See HODINKEE At The Pop-Up Flea, This Wee...",HODINKEE,2013-12-06 14:00:00-05:00,2017-02-20 21:05:29-05:00,-42,390
2,2,Not Specified,On The Block: Highlights From The December 17t...,HODINKEE,2013-12-06 11:28:26-05:00,2017-02-20 21:06:00-05:00,-42,607
3,3,Hands-On,With The New A. Lange & Sohne Grand Lange 1 Mo...,Blake Buettner,2013-12-05 21:53:45-05:00,2017-02-20 21:08:09-05:00,-42,376
4,4,Not Specified,Pre-SIHH 2014: Introducing The A. Lange & Söhn...,HODINKEE,2013-12-05 12:28:27-05:00,2017-02-20 21:08:08-05:00,-42,353
...,...,...,...,...,...,...,...,...
6496,6530,Best of Watchville,Remembering The Ikepod Megapode,Jon Bues,2020-04-03 11:00:22-04:00,2020-04-03 11:00:25-04:00,5,259
6497,6531,The Daily Uplift,Foxfire And Friday Live Gets A Futuristic Disc...,"James Stacey,",2020-04-03 12:00:07-04:00,2020-04-03 12:00:11-04:00,5,569
6498,6532,Friday Live,The Return,Cara Barrett,2020-04-03 12:55:00-04:00,2020-04-04 15:20:31-04:00,26,167
6499,6533,Sunday Rewind,How The (Tudor) Straps Are Made,HODINKEE,2020-04-05 09:00:03-04:00,2020-04-05 09:00:08-04:00,18,169


In [117]:
# datetime.strptime(q_c['comment_published'][0][:-5], '%Y-%m-%dT%H:%M:%S')
# datetime.strptime(df_metadata['article_published'][q_c['article_num'][0]][:-5], '%Y-%m-%d %H:%M:%S-')

for i, t in enumerate(q_c['comment_published']):
    print(i, t, q_c['article_num'][i])
    if (datetime.strptime(t[:-5], '%Y-%m-%dT%H:%M:%S') - datetime.strptime(df_metadata['article_published'][q_c['article_num'][i]][:-5], '%Y-%m-%d %H:%M:%S-')) > timedelta(1209600):
        print(True)

0 2017-01-11T16:55:57.000Z 3605
1 2017-01-11T16:58:41.000Z 3605
2 2017-01-11T17:22:05.000Z 3605
3 2017-01-11T17:41:34.000Z 3605
4 2017-01-11T18:42:25.000Z 3605
5 2017-01-11T18:52:46.000Z 3605
6 2017-01-11T18:59:32.000Z 3605
7 2017-01-11T19:00:24.000Z 3605
8 2017-01-11T19:35:01.000Z 3605
9 2017-01-11T19:56:02.000Z 3610
10 2017-01-11T20:30:19.000Z 3605
11 2017-01-11T20:36:19.000Z 3610
12 2017-01-11T20:49:27.000Z 3610
13 2017-01-11T22:26:42.000Z 3610
14 2017-01-11T23:12:10.000Z 3605
15 2017-01-11T23:19:05.000Z 3612
16 2017-01-11T23:26:44.000Z 3612
17 2017-01-12T00:03:37.000Z 3612
18 2017-01-12T01:01:05.000Z 3612
19 2017-01-12T02:24:13.000Z 3612
20 2017-01-12T04:57:54.000Z 3612
21 2017-01-12T06:10:12.000Z 3612
22 2017-01-12T07:42:04.000Z 3612
23 2017-01-12T12:05:11.000Z 3605
24 2017-01-12T13:29:39.000Z 3612
25 2017-01-12T16:25:36.000Z 3607
26 2017-01-12T18:09:15.000Z 3614
27 2017-01-12T19:05:04.000Z 3597
28 2017-01-12T19:22:35.000Z 3597
29 2017-01-12T20:35:32.000Z 3597
30 2017-01-13T04:33:

KeyError: 6512

In [99]:
# 14 days is 1,209,600 seconds

In [3]:
titles = df_metadata['article_title']

In [4]:
q_comments = df_comments['comment_text']

In [78]:
q_comments.str.contains('\?\!')
count=0
for i in q_comments:
    if '?!' in i:
        print(i, '\n')
        count+=1
count

What does this guy do for a living?! 

And did I hear him say Queens, NY? 

Bayside/Little Neck over here! 

Can somebody please tell me where those beautiful cases are from?! 

I wonder what's the profit margin on this thing?!? 

First thoughts, Longines - where is the date window ?! This is beautifully done. 

Dials remind me of H.Moser, however the date window irks (why is there a temptation to add a date window ?!)

I would have loved to see some contrast on the movement decoration...just wishes ;-) 

Lange & AP. Everyday watches?! 

Agreed! Omega Chronos really have a special place in my heart! These pieces are timeless and can take a beating! Hey, they've been tested in outer space!!!! What can you possibly do to them on earth?! 

If you have unlimited funds, a million dollar watch can be an everyday watch. Why the hell not?! It just depends on your bank account. It's just like cars...some people drive VW's as every day cars, while some drive a Rolls Royce as their every day car.

801

1. Tokenize
2. Set to lowercase
3. Remove stopped words
4. Stem
5. Vectorizer fit to trivia questions
6. Transform question-comments using vectorizer
7. Transform titles using vectorizer
8. Calculate cosine similarity for question-comments and a. trivia questions; b. article titles
9. Evaluate pairs with perfect similarity scores

## Prep to Vectorize

In [68]:
def pre_vec(q_list):
    pre_vec = []
    
    for q in q_list:
        q = prep.tokenize(q)
        q = prep.lowercase(q)
#         q = prep.rem_stop(q)
        q = prep.stem(q)
        q = ' '.join(q)
        
        pre_vec.append(q)
        
    return pre_vec

In [69]:
titles_pre_vec = pre_vec(titles)
qc_pre_vec = pre_vec(q_comments)

## Vectorize

### TF-IDF

In [70]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()

In [71]:
qc_tfidf = tfidf.fit_transform(qc_pre_vec)

titles_tfidf = tfidf.transform(titles_pre_vec)

In [72]:
vocab_tfidf = tfidf.vocabulary_
print(len(vocab_tfidf), type(vocab_tfidf))

20105 <class 'dict'>


### Counts

In [73]:
counts = text.CountVectorizer()

In [74]:
qc_counts = counts.fit_transform(qc_pre_vec)

titles_counts = counts.transform(titles_pre_vec)

In [75]:
vocab = counts.vocabulary_
print(len(vocab), type(vocab))

20105 <class 'dict'>


### Word2Vec

## Cosine Similarities

In [76]:
sim_counts = pd.DataFrame(cosine_similarity(qc_tfidf, titles_tfidf))

In [31]:
sim_counts.describe()

,0,1,2,3,4,5,6,7,8,9,...,6491,6492,6493,6494,6495,6496,6497,6498,6499,6500
count,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,...,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000
mean,0.010701,0.009367,0.004046,0.008931,0.004237,0.004214,0.004676,0.006201,0.012577,0.003802,...,0.007945,0.005524,0.006441,0.002857,0.004846,0.001029,0.003441,0.000733,0.010069,0.000633
std,0.028899,0.021999,0.016515,0.027461,0.023120,0.015202,0.015660,0.020567,0.030597,0.017281,...,0.020198,0.018077,0.025199,0.017155,0.018722,0.011582,0.014452,0.014012,0.038950,0.007694
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.389429,0.453517,0.399001,0.551729,0.524797,0.368106,0.405661,0.294826,0.465648,0.498994,...,0.380016,0.308118,0.474864,0.612766,0.452153,0.522702,0.349070,1.000000,0.586449,0.242491


In [80]:
count = 0

for i in sim_counts:
    if sim_counts[i].max() > 0.8:
        print(sim_counts[i].max())
        print(df_comments.loc[sim_counts[i].idxmax()])
        print(df_metadata.loc[i], '\n\n')
        
        count+=1
        
print(count)

0.8048369462785332
Unnamed: 0                              24038
article_num                              4293
comment_id                              26575
commenter_name                       duckjibe
comment_published    2017-10-04T11:45:28.000Z
comment_text                         Hublot ?
comment_likes                               0
comment_flag                                0
parent_id                                   0
Name: 4983, dtype: object
article_id                                                         719
article_cat                                              Not Specified
article_title        The Hublot Atelier: The Hublot for when your o...
article_author                                            Felix Scholz
article_published                            2012-10-03 09:00:00-04:00
article_modified                             2017-02-20 21:01:21-05:00
comm_count                                                         -42
word_count                                  

In [82]:
titles[4428]

'The Plural Of Rolex'

In [84]:
sim_counts[4428].idxmax()

6531

In [87]:
df_comments.loc[6531]

Unnamed: 0                                                       31635
article_num                                                       4428
comment_id                                                       34647
commenter_name                                               multanemo
comment_published                             2017-12-28T13:52:03.000Z
comment_text         The plural of Rolex at an AD when you ask for ...
comment_likes                                                        2
comment_flag                                                         0
parent_id                                                            0
Name: 6531, dtype: object

In [ ]:
sim_tf = pd.DataFrame(cosine_similarity(qc_tfidf, qt_tfidf))

In [ ]:
sim_tf.describe()

In [ ]:
max_c = []
max_c_scores = []

for index, i in enumerate(sims.columns):
    max_c.append(q_comments[sims[i].idxmax()])
    max_c_scores.append(sims[i].max())

In [ ]:
for index, i in enumerate(max_c):
    print(trivia_questions[index])
    print(i)
    print('\n')